In [1]:
from pytket.circuit import Circuit, CircBox, QControlBox

from pytket.circuit.display import render_circuit_jupyter

from pytket.extensions.nexus import NexusBackend, QuantinuumConfig, Nexus
from datetime import datetime

from pytket.backends.backendresult import BackendResult

import phayes

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Prepares circuit by hadamarding all the qubits
def prepare_circ(circ: Circuit) -> Circuit():
    n_qubits = circ.n_qubits
    prepped_circ = Circuit(n_qubits)
    
    for i in range(n_qubits):
        prepped_circ.H(i)
    return prepped_circ

In [14]:
# Constructs unitary for an arbitrary size of qubits
def construct_unitary(n_qubits: int, theta) -> Circuit():
    ### Wait on Carson and Christine ###
    u_circ = Circuit(n_qubits)
    for i in range(n_qubits):
        u_circ.U1(theta, i)
    return u_circ

In [ ]:
def bayes_qpe_iteration(prepped_circ: Circuit, cu_circ: Circuit,  beta: int, k_factor: int):
    prepped_circ.add

In [15]:
def bayes_qpe(prepped_circ: Circuit, cu_circ: Circuit, theta) -> Circuit():
    n_qubits = prepped_circ.n_qubits
    qpe_circ = Circuit(n_qubits + 1)
    
    return qpe_circ

test_qubits = 4
theta = np.pi / 4

test_circ = Circuit(test_qubits)
prepped_circ = prepare_circ(test_circ)

test_u = construct_unitary(test_qubits, theta)
qpe_circ = bayes_qpe(prepped_circ, test_u, 0)

render_circuit_jupyter(qpe_circ)